In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from loguru import logger

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.sparse as sparse
import torch.optim as optim

import pandas as pd
import plotly.express as px

sys.path.insert(0, '..')

from src.viz import blueq_colors

# Implement

In [3]:
from src.train_utils import mse_loss, train, MetricLogCallback
from src.model import LightGCN

In [4]:
# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )
device = 'cpu'
logger.info(f"Using {device} device")

2024-09-09 00:07:12.291 | INFO     | __main__:<module>:9 - Using cpu device


# Test implementation

In [5]:
# Mock data
user_ids = [0, 0, 1, 2, 2]
item_ids = [0, 1, 2, 3, 1]
ratings = [1, 4, 5, 3, 2]
n_users = len(set(user_ids))
n_items = len(set(item_ids))

val_user_ids = [0, 1, 2]
val_item_ids = [2, 1, 2]
val_ratings = [2, 4, 5]

model = LightGCN(embedding_dim=64, n_layers=3, user_ids=user_ids, item_ids=item_ids, ratings=ratings, device=device)

# Example forward pass
users = torch.tensor([0, 1, 2])
items = torch.tensor([0, 1, 2])
predictions = model.predict(users, items)
print(predictions)

tensor([-0.0043,  0.0017, -0.0015], grad_fn=<SumBackward1>)


In [6]:
import random
import numpy as np
from torch.utils.data import DataLoader
from src.dataset_loader import UserItemRatingDataset

# Fixing random seed for reproducibility
random.seed(42)
np.random.seed(42)

embedding_dim = 8
n_layers = 3
batch_size = 4

# Display mock dataset
print("Mock User IDs:", user_ids)
print("Mock Item IDs:", item_ids)
print("Ratings:", ratings)

# class RatingDataset(Dataset):
#     def __init__(self, user_ids, item_ids, ratings):
#         """
#         Args:
#             user_ids (list or array): List of user indices.
#             item_ids (list or array): List of item indices.
#             ratings (list or array): List of corresponding ratings.
#         """
#         self.user_ids = user_ids
#         self.item_ids = item_ids
#         self.ratings = ratings
    
#     def __len__(self):
#         return len(self.user_ids)
    
#     def __getitem__(self, idx):
#         user = self.user_ids[idx]
#         item = self.item_ids[idx]
#         rating = self.ratings[idx]
#         return user, item, rating

rating_dataset = UserItemRatingDataset(user_ids, item_ids, ratings)
dataloader = DataLoader(rating_dataset, batch_size=batch_size, shuffle=True)

val_rating_dataset = UserItemRatingDataset(val_user_ids, val_item_ids, val_ratings)
val_dataloader = DataLoader(val_rating_dataset, batch_size=batch_size, shuffle=True)

# Instantiate LightGCN model
model = LightGCN(embedding_dim=embedding_dim, n_layers=n_layers,
                 user_ids=user_ids, item_ids=item_ids, ratings=ratings, device=device)

# Training loop
n_epochs = 50

train(model, dataloader, val_dataloader, epochs=n_epochs, patience=2, print_steps=1, lr=0.001, device=device, progress_bar_type='tqdm_notebook')

Mock User IDs: [0, 0, 1, 2, 2]
Mock Item IDs: [0, 1, 2, 3, 1]
Ratings: [1, 4, 5, 3, 2]


Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Training Epoch 1:   0%|          | 0/2 [00:00<?, ?it/s]

2024-09-09 00:07:12.785 | INFO     | src.train_utils:train:122 - Step 1, Global Loss: 11.5151
2024-09-09 00:07:12.786 | INFO     | src.train_utils:train:125 - Step 1, Learning Rate: 0.001000
2024-09-09 00:07:12.786 | INFO     | src.train_utils:train:128 - Step 1, Gradient Norms: {'grad_norm_user_embedding.weight': 0.06376536190509796, 'grad_norm_item_embedding.weight': 0.07702846080064774, 'total_grad_norm': 0.0999970257167934}
2024-09-09 00:07:12.788 | INFO     | src.train_utils:train:122 - Step 2, Global Loss: 10.2530
2024-09-09 00:07:12.788 | INFO     | src.train_utils:train:125 - Step 2, Learning Rate: 0.001000
2024-09-09 00:07:12.789 | INFO     | src.train_utils:train:128 - Step 2, Gradient Norms: {'grad_norm_user_embedding.weight': 0.11588092893362045, 'grad_norm_item_embedding.weight': 0.11571361124515533, 'total_grad_norm': 0.16376211258381387}
2024-09-09 00:07:12.789 | INFO     | src.train_utils:train:140 - Epoch 1, Loss: 10.2530
2024-09-09 00:07:12.790 | INFO     | src.train_

Training Epoch 2:   0%|          | 0/2 [00:00<?, ?it/s]

2024-09-09 00:07:12.795 | INFO     | src.train_utils:train:122 - Step 3, Global Loss: 13.5106
2024-09-09 00:07:12.795 | INFO     | src.train_utils:train:125 - Step 3, Learning Rate: 0.001000
2024-09-09 00:07:12.796 | INFO     | src.train_utils:train:128 - Step 3, Gradient Norms: {'grad_norm_user_embedding.weight': 0.07017427682876587, 'grad_norm_item_embedding.weight': 0.08113785833120346, 'total_grad_norm': 0.10727432676560925}
2024-09-09 00:07:12.797 | INFO     | src.train_utils:train:122 - Step 4, Global Loss: 7.2579
2024-09-09 00:07:12.797 | INFO     | src.train_utils:train:125 - Step 4, Learning Rate: 0.001000
2024-09-09 00:07:12.797 | INFO     | src.train_utils:train:128 - Step 4, Gradient Norms: {'grad_norm_user_embedding.weight': 0.04914429411292076, 'grad_norm_item_embedding.weight': 0.05210519954562187, 'total_grad_norm': 0.07162481039099743}
2024-09-09 00:07:12.798 | INFO     | src.train_utils:train:140 - Epoch 2, Loss: 7.2579
2024-09-09 00:07:12.799 | INFO     | src.train_u

Training Epoch 3:   0%|          | 0/2 [00:00<?, ?it/s]

2024-09-09 00:07:12.803 | INFO     | src.train_utils:train:122 - Step 5, Global Loss: 12.7595
2024-09-09 00:07:12.803 | INFO     | src.train_utils:train:125 - Step 5, Learning Rate: 0.001000
2024-09-09 00:07:12.803 | INFO     | src.train_utils:train:128 - Step 5, Gradient Norms: {'grad_norm_user_embedding.weight': 0.06755920499563217, 'grad_norm_item_embedding.weight': 0.07502468675374985, 'total_grad_norm': 0.10096013966977334}
2024-09-09 00:07:12.804 | INFO     | src.train_utils:train:122 - Step 6, Global Loss: 8.3827
2024-09-09 00:07:12.805 | INFO     | src.train_utils:train:125 - Step 6, Learning Rate: 0.001000
2024-09-09 00:07:12.805 | INFO     | src.train_utils:train:128 - Step 6, Gradient Norms: {'grad_norm_user_embedding.weight': 0.06795651465654373, 'grad_norm_item_embedding.weight': 0.07804109156131744, 'total_grad_norm': 0.10348188177814982}
2024-09-09 00:07:12.805 | INFO     | src.train_utils:train:140 - Epoch 3, Loss: 8.3827
2024-09-09 00:07:12.806 | INFO     | src.train_u

# Prep data

In [7]:
train_df = pd.read_parquet("../data/train.parquet")
val_df = pd.read_parquet("../data/val.parquet")

In [8]:
from src.id_mapper import IDMapper

In [9]:
user_ids = train_df['user_id'].values
item_ids = train_df['parent_asin'].values
unique_user_ids = list(set(user_ids))
unique_item_ids = list(set(item_ids))

logger.info(f"{len(unique_user_ids)=:,.0f}, {len(unique_item_ids)=:,.0f}")

2024-09-09 00:07:12.882 | INFO     | __main__:<module>:6 - len(unique_user_ids)=5,223, len(unique_item_ids)=2,653


In [10]:
idm = IDMapper()
idm.fit(unique_user_ids, unique_item_ids)

In [11]:
user_indices = [idm.get_user_index(user_id) for user_id in user_ids]
item_indices = [idm.get_item_index(item_id) for item_id in item_ids]
ratings = train_df['rating'].values.tolist()

In [12]:
val_user_indices = [idm.get_user_index(user_id) for user_id in val_df['user_id']]
val_item_indices = [idm.get_item_index(item_id) for item_id in val_df['parent_asin']]
val_ratings = val_df['rating'].values.tolist()

# Train

In [13]:
batch_size = 32

rating_dataset = UserItemRatingDataset(user_indices, item_indices, ratings)
dataloader = DataLoader(rating_dataset, batch_size=batch_size, shuffle=True)

val_rating_dataset = UserItemRatingDataset(val_user_indices, val_item_indices, val_ratings)
val_dataloader = DataLoader(val_rating_dataset, batch_size=batch_size, shuffle=True)

In [14]:
embedding_dim = 128
n_layers = 3

# Instantiate LightGCN model
model = LightGCN(embedding_dim=embedding_dim, n_layers=n_layers,
                 user_ids=user_indices, item_ids=item_indices, ratings=ratings, device=device)

#### Predict before train

In [15]:
user_id = 'AEHW2B54HDLZ3APBEWXHYLZ6SSYQ'
val_df.loc[lambda df: df['user_id'].eq(user_id)]

,user_id,parent_asin,rating,timestamp
34367,AEHW2B54HDLZ3APBEWXHYLZ6SSYQ,B07MYVF61Y,4.0,1654225907045


In [16]:
item_id = 'B07MYVF61Y'
user_indice = idm.get_user_index(user_id)
item_indice = idm.get_item_index(item_id)

model.predict([user_indice], [item_indice])

tensor([0.0066], grad_fn=<SumBackward1>)

#### Training loop

In [ ]:
n_epochs = 50

metric_log_cb = MetricLogCallback()

train(
    model,
    dataloader,
    val_dataloader,
    epochs=n_epochs,
    patience=2,
    print_steps=100,
    lr=0.03,
    device=device,
    progress_bar_type='tqdm_notebook',
    callbacks=[metric_log_cb.process_payload]
)

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Training Epoch 1:   0%|          | 0/566 [00:00<?, ?it/s]

2024-09-09 00:07:14.880 | INFO     | src.train_utils:train:122 - Step 100, Global Loss: 19.7837
2024-09-09 00:07:14.881 | INFO     | src.train_utils:train:125 - Step 100, Learning Rate: 0.030000
2024-09-09 00:07:14.881 | INFO     | src.train_utils:train:128 - Step 100, Gradient Norms: {'grad_norm_user_embedding.weight': 0.2209496945142746, 'grad_norm_item_embedding.weight': 0.17535638809204102, 'total_grad_norm': 0.2820791207279223}
2024-09-09 00:07:16.753 | INFO     | src.train_utils:train:122 - Step 200, Global Loss: 19.6663
2024-09-09 00:07:16.753 | INFO     | src.train_utils:train:125 - Step 200, Learning Rate: 0.030000
2024-09-09 00:07:16.754 | INFO     | src.train_utils:train:128 - Step 200, Gradient Norms: {'grad_norm_user_embedding.weight': 0.42739802598953247, 'grad_norm_item_embedding.weight': 0.3074932098388672, 'total_grad_norm': 0.5265179452941359}
2024-09-09 00:07:18.665 | INFO     | src.train_utils:train:122 - Step 300, Global Loss: 19.3867
2024-09-09 00:07:18.666 | INFO

Training Epoch 2:   0%|          | 0/566 [00:00<?, ?it/s]

2024-09-09 00:07:25.383 | INFO     | src.train_utils:train:122 - Step 600, Global Loss: 6.5678
2024-09-09 00:07:25.384 | INFO     | src.train_utils:train:125 - Step 600, Learning Rate: 0.030000
2024-09-09 00:07:25.384 | INFO     | src.train_utils:train:128 - Step 600, Gradient Norms: {'grad_norm_user_embedding.weight': 0.3292803168296814, 'grad_norm_item_embedding.weight': 0.32689398527145386, 'total_grad_norm': 0.4639883669426518}
2024-09-09 00:07:27.395 | INFO     | src.train_utils:train:122 - Step 700, Global Loss: 6.1948
2024-09-09 00:07:27.395 | INFO     | src.train_utils:train:125 - Step 700, Learning Rate: 0.030000
2024-09-09 00:07:27.396 | INFO     | src.train_utils:train:128 - Step 700, Gradient Norms: {'grad_norm_user_embedding.weight': 0.42932167649269104, 'grad_norm_item_embedding.weight': 0.20987744629383087, 'total_grad_norm': 0.4778761810022704}
2024-09-09 00:07:29.430 | INFO     | src.train_utils:train:122 - Step 800, Global Loss: 5.7125
2024-09-09 00:07:29.431 | INFO  

...

2024-09-09 00:08:41.862 | INFO     | src.train_utils:train:122 - Step 4000, Global Loss: 1.2495
2024-09-09 00:08:41.863 | INFO     | src.train_utils:train:125 - Step 4000, Learning Rate: 0.019683
2024-09-09 00:08:41.863 | INFO     | src.train_utils:train:128 - Step 4000, Gradient Norms: {'grad_norm_user_embedding.weight': 0.2811598479747772, 'grad_norm_item_embedding.weight': 0.20301806926727295, 'total_grad_norm': 0.3467956120861553}
2024-09-09 00:08:43.999 | INFO     | src.train_utils:train:122 - Step 4100, Global Loss: 1.1203
2024-09-09 00:08:44.000 | INFO     | src.train_utils:train:125 - Step 4100, Learning Rate: 0.019683
2024-09-09 00:08:44.000 | INFO     | src.train_utils:train:128 - Step 4100, Gradient Norms: {'grad_norm_user_embedding.weight': 0.2328053116798401, 'grad_norm_item_embedding.weight': 0.17440468072891235, 'total_grad_norm': 0.29088710147839375}
2024-09-09 00:08:46.189 | INFO     | src.train_utils:train:122 - Step 4200, Global Loss: 1.1176
2024-09-09 00:08:46.190 |

Training Epoch 9:   0%|          | 0/566 [00:00<?, ?it/s]

2024-09-09 00:08:55.635 | INFO     | src.train_utils:train:122 - Step 4600, Global Loss: 0.9214
2024-09-09 00:08:55.636 | INFO     | src.train_utils:train:125 - Step 4600, Learning Rate: 0.019683
2024-09-09 00:08:55.636 | INFO     | src.train_utils:train:128 - Step 4600, Gradient Norms: {'grad_norm_user_embedding.weight': 0.20199033617973328, 'grad_norm_item_embedding.weight': 0.14372995495796204, 'total_grad_norm': 0.24790803912382403}
2024-09-09 00:08:57.808 | INFO     | src.train_utils:train:122 - Step 4700, Global Loss: 0.9491
2024-09-09 00:08:57.808 | INFO     | src.train_utils:train:125 - Step 4700, Learning Rate: 0.019683
2024-09-09 00:08:57.808 | INFO     | src.train_utils:train:128 - Step 4700, Gradient Norms: {'grad_norm_user_embedding.weight': 0.16073039174079895, 'grad_norm_item_embedding.weight': 0.12258189916610718, 'total_grad_norm': 0.2021400030481853}
2024-09-09 00:08:59.936 | INFO     | src.train_utils:train:122 - Step 4800, Global Loss: 0.9713
2024-09-09 00:08:59.936

Training Epoch 10:   0%|          | 0/566 [00:00<?, ?it/s]

2024-09-09 00:09:07.239 | INFO     | src.train_utils:train:122 - Step 5100, Global Loss: 0.7466
2024-09-09 00:09:07.239 | INFO     | src.train_utils:train:125 - Step 5100, Learning Rate: 0.017715
2024-09-09 00:09:07.240 | INFO     | src.train_utils:train:128 - Step 5100, Gradient Norms: {'grad_norm_user_embedding.weight': 0.22143200039863586, 'grad_norm_item_embedding.weight': 0.14372944831848145, 'total_grad_norm': 0.2639891761312886}
2024-09-09 00:09:09.451 | INFO     | src.train_utils:train:122 - Step 5200, Global Loss: 0.8400
2024-09-09 00:09:09.452 | INFO     | src.train_utils:train:125 - Step 5200, Learning Rate: 0.017715
2024-09-09 00:09:09.452 | INFO     | src.train_utils:train:128 - Step 5200, Gradient Norms: {'grad_norm_user_embedding.weight': 0.18381869792938232, 'grad_norm_item_embedding.weight': 0.1477144956588745, 'total_grad_norm': 0.23581536408005557}
2024-09-09 00:09:11.594 | INFO     | src.train_utils:train:122 - Step 5300, Global Loss: 0.8467
2024-09-09 00:09:11.595 

# Visualize training

In [ ]:
step_metrics = [p for p in metric_log_cb.payloads if 'step' in p]
epoch_metrics = [p for p in metric_log_cb.payloads if 'epoch' in p]
step_metrics_df = pd.DataFrame(step_metrics)
step_metrics_df

In [ ]:
epoch_metrics_df = pd.DataFrame(epoch_metrics)
epoch_metrics_df = epoch_metrics_df.pipe(lambda df: pd.melt(df, id_vars=["epoch"], var_name="loss_type", value_name="value"))
epoch_metrics_df

In [ ]:
def plot_metric(df, index='step', col: str = None, color=None):
    # Create the plot
    cols = [index, col]
    if color:
        cols.append(color)
    fig = px.line(
        df[cols].dropna(),
        x=index,
        y=col,
        color=color,
        labels={'x': index, 'y': col},
        title=f'{col} by {index}',
    )

    if color:
        fig.update_layout(showlegend=True)

    fig.show()

plot_metric(step_metrics_df, col='global_loss')
plot_metric(step_metrics_df, col='learning_rate')
plot_metric(step_metrics_df, col='total_grad_norm')
plot_metric(epoch_metrics_df, index='epoch', col='value', color='loss_type')

In [ ]:
fig = px.bar(
    step_metrics_df,
    x='step',
    y=['grad_norm_user_embedding.weight', 'grad_norm_item_embedding.weight'],
    title='Norm gradients of user embeddings and item embeddings',
    height=500
)
fig.update_layout(showlegend=True, barmode='stack')
fig.show()

# Predict

In [ ]:
train_df

In [ ]:
user_id = 'AEHW2B54HDLZ3APBEWXHYLZ6SSYQ'
val_df.loc[lambda df: df['user_id'].eq(user_id)]

In [ ]:
item_id = 'B07MYVF61Y'
user_indice = idm.get_user_index(user_id)
item_indice = idm.get_item_index(item_id)

model.predict([user_indice], [item_indice])